In [1]:
## Imports

# From standard library
import json

# From third party
import boto3
import botocore
from dotenv import dotenv_values

In [66]:
# IAM Role Global Variables
PATH = "/"
ROLE_NAME = "dwhRole"
DESCRIPTION = "Allows Redshift clusters to call AWS services on your behalf."
ASSUME_ROLE_POLICY_DOCUMENT = json.dumps(
    {
        "Statement": [
            {
                "Action": "sts:AssumeRole",
                "Effect": "Allow",
                "Principal": {"Service": "redshift.amazonaws.com"},
            }
        ],
        "Version": "2012-10-17",
    }
)
POLICY_NAME = "AmazonS3ReadOnlyAccess"
POLICY_ARN = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"  # For attaching policies

# Redshift Global Variables
CLUSTER_IDENTIFIER = "dwhCluster"
CLUSTER_TYPE = "multi-node"
NUM_NODES = 4
NODE_TYPE = "dc2.large"
DB_NAME = "dwh"
MASTER_USER_NAME = "dwhuser"
MASTER_USER_PASSWORD = "Passw0rd"

# Connection Global Variables
PORT = 5439
CIDR_IP = "0.0.0.0/0"
IP_PROTOCOL = "TCP"

In [3]:
# Get AWS credentials from .env file
env = dotenv_values()

AWS_KEY = env["AWS_KEY"]
AWS_SECRET = env["AWS_SECRET"]

In [7]:
s3 = boto3.resource("s3", region_name="us-west-2", aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
ec2 = boto3.client("ec2", region_name="us-west-2", aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
iam = boto3.client("iam", region_name="us-west-2", aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
redshift = boto3.client("redshift", region_name="us-west-2", aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)

In [62]:
def role_exists(iam, role_name):
    try:
        iam.get_role(RoleName=role_name)
        return True
    except iam.exceptions.NoSuchEntityException:
        return False
    
def role_policy_exists(iam, role_name, policy_arn):
    try:
        iam.get_role_policy(RoleName=role_name, PolicyName=POLICY_NAME)
        return True
    except iam.exceptions.NoSuchEntityException:
        return False
    except Exception as e:
        print(e)
        return False
    
role_exists(iam, ROLE_NAME)

True

In [ ]:
def create_role(iam, role_name: str = ROLE_NAME, description: str = DESCRIPTION, assume_role_policy_document: str = ASSUME_ROLE_POLICY_DOCUMENT):
    if role_exists(iam, role_name):
        print("Role already exists")
        return
    else:
        iam.create_role(
            Path="/",
            RoleName=ROLE_NAME,
            Description=DESCRIPTION,
            AssumeRolePolicyDocument=ASSUME_ROLE_POLICY_DOCUMENT,
        )

In [65]:
role_policy_exists(iam, ROLE_NAME, "AmazonS3ReadOnlyAccess")

False

In [53]:
try:
    role_arn = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]
except iam.exceptions.NoSuchEntityException as e:
    iam.create_role(
        Path="/",
        RoleName=ROLE_NAME,
        Description=DESCRIPTION,
        AssumeRolePolicyDocument=ASSUME_ROLE_POLICY_DOCUMENT,
    )
    role_arn = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]

In [59]:
try:
    iam.detach_role_policy(
        RoleName=ROLE_NAME, 
        PolicyArn=POLICY_ARN
    )
except iam.exceptions.NoSuchEntityException as e:
    print(e)

An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: Policy arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess was not found.


In [45]:
def get_dir(obj):
    return [item for item in dir(obj) if not item.startswith("_")]

get_dir(iam)

['add_client_id_to_open_id_connect_provider',
 'add_role_to_instance_profile',
 'add_user_to_group',
 'attach_group_policy',
 'attach_role_policy',
 'attach_user_policy',
 'can_paginate',
 'change_password',
 'close',
 'create_access_key',
 'create_account_alias',
 'create_group',
 'create_instance_profile',
 'create_login_profile',
 'create_open_id_connect_provider',
 'create_policy',
 'create_policy_version',
 'create_role',
 'create_saml_provider',
 'create_service_linked_role',
 'create_service_specific_credential',
 'create_user',
 'create_virtual_mfa_device',
 'deactivate_mfa_device',
 'delete_access_key',
 'delete_account_alias',
 'delete_account_password_policy',
 'delete_group',
 'delete_group_policy',
 'delete_instance_profile',
 'delete_login_profile',
 'delete_open_id_connect_provider',
 'delete_policy',
 'delete_policy_version',
 'delete_role',
 'delete_role_permissions_boundary',
 'delete_role_policy',
 'delete_saml_provider',
 'delete_server_certificate',
 'delete_servic

In [19]:
with open('../dwh.cfg') as f:
    config.read_file(f)

In [22]:
config["CLUSTER"]["Hostility"] = "blub"

In [20]:
config["IAM_ROLE"]["ARN"] = "bla"

In [15]:
config.add_section("SOMETHING")

In [23]:
with open('../dwh.cfg', 'w') as file:
    config.write(file)